In [2]:
from langgraph.graph import StateGraph, START , END
from typing import TypedDict, Annotated , List
from langchain_core.messages import BaseMessage, HumanMessage, SystemMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_core.tools import tool
from langgraph.types import interrupt, Command
from dotenv import load_dotenv
import requests
from pathlib import Path
import operator
from pydantic import BaseModel, Field
from typing import Literal
from langchain_mistralai import ChatMistralAI
from langgraph.types import Send
import re
import os

key = "N56JL9JhomtYzc64dQh7K05t6m0G8u9Q"

model = ChatMistralAI(
    model="mistral-large-2512",
    mistral_api_key= key,  # Direct key
    temperature=0.7
)

In [3]:
class Task(BaseModel):
    id : int 
    title : str 
    brief : str = Field(..., description= " What to cover")
    Systemprompt : str = Field(..., description= "Generate a system prompt for each task.")

Now we get system prompt for each task by llm.

In [4]:
class Plan(BaseModel):
    blog_title : str 
    tasks : List[Task]

In [5]:
class State(TypedDict):
    topic : str 
    plan : Plan 
    #reducer : results from workers can get connected automatically 
    Section : Annotated[List[str], operator.add]
    final_blog : str 

In [6]:
def orchestrator(State : State):
    llm_with_structure_output = model.with_structured_output(Plan)
    plan = llm_with_structure_output.invoke([
        SystemMessage(content="Create blog with 5-7 sections on the following topic"),
        HumanMessage(content= f"Topic : {State["topic"]}")
    ])

    return {"plan" : plan}



In [7]:
def condition_toget_workers(State : State):
    return [Send("worker", {"task": task, "topic" : State["topic"], "plan": State["plan"]}) for task in State["plan"].tasks]

In [8]:
def worker(payload : dict) -> dict:
    task = payload["task"]
    topic = payload["topic"]
    plan = payload["plan"]
    blog_title = plan.blog_title

    section = [SystemMessage(content= f"{task.Systemprompt}"),
               HumanMessage(content = 
                            f"""Blog : {blog_title}\n
                            topic : {topic}\n\ng
                            Section : {task.title}\n
                            Breif : {task.brief}
                            Return only the section content in Markdown""")]
    response = model.invoke(section).content.strip()
    return {"Section" : [response]}

In [9]:
def reducer(State: State) -> dict:
    
    title = State["plan"].blog_title
    body = "\n\n".join(State["Section"]).strip()

    final_md = f"# {title}\n\n{body}\n"

    # ---- save to file ----
    filename = "research_paper4.md"
    output_path = Path(filename)
    output_path.write_text(final_md, encoding="utf-8")

    return {"final_blog": final_md}

In [10]:
graph = StateGraph(State)

In [11]:
graph.add_node("orchestrator", orchestrator)
graph.add_node("worker",worker)
graph.add_node("reducer",reducer)

graph.add_edge(START,"orchestrator")
graph.add_conditional_edges("orchestrator", condition_toget_workers, ["worker"])
graph.add_edge("worker","reducer")
graph.add_edge("reducer",END)

workflow = graph.compile()

In [ ]:
result = workflow.invoke({"topic": "","Section": []})

In [13]:
result 

{'topic': 'Planning Agents',
 'plan': Plan(blog_title='The Ultimate Guide to Planning Agents: Revolutionizing Automation and Decision-Making', tasks=[Task(id=1, title='Introduction to Planning Agents', brief='Define what planning agents are, their origin, and their significance in modern technology. Explain how they differ from traditional software agents.', Systemprompt='Write an engaging and informative introduction to planning agents. Define what planning agents are, their origin, and their evolution in the field of artificial intelligence. Explain how they differ from traditional software agents and why they are significant in modern technology. Ensure the tone is accessible to both technical and non-technical readers.'), Task(id=2, title='How Planning Agents Work', brief='Dive into the mechanics of planning agents, including their core components like goal setting, action sequencing, and environment modeling. Explain concepts like state space, search algorithms, and heuristics.', 

In [88]:
result 

{'topic': 'Self Attention architecture',
 'plan': Plan(blog_title='Understanding Self-Attention Architecture: The Backbone of Modern NLP', tasks=[Task(id=1, title='Introduction to Self-Attention Architecture', brief='Explain what self-attention architecture is, its importance in modern NLP, and why it revolutionized the field.', Systemprompt='Write an engaging introduction to self-attention architecture. Explain what it is, its significance in the field of NLP, and how it has revolutionized tasks like machine translation, text summarization, and more. Use simple language to make it accessible to beginners.'), Task(id=2, title='The Evolution of Attention Mechanisms', brief='Provide a brief history of attention mechanisms, starting from their introduction in RNNs and LSTMs to the development of self-attention.', Systemprompt='Create a section that traces the evolution of attention mechanisms. Start with their introduction in RNNs and LSTMs, explain their limitations, and then introduce h

In [16]:
result["plan"]


Plan(blog_title='Understanding Self-Attention Architecture: The Backbone of Modern NLP', tasks=[Task(id=1, title='Introduction to Self-Attention', brief='Explain what self-attention is, its importance in modern NLP, and how it differs from traditional architectures like RNNs and CNNs. Provide a high-level overview of its applications.'), Task(id=2, title='The Mathematics Behind Self-Attention', brief='Dive into the mathematical foundations of self-attention. Cover concepts like Query, Key, and Value matrices, dot-product attention, and the softmax function. Use simple examples to illustrate the calculations.'), Task(id=3, title='Why Self-Attention? Advantages and Limitations', brief='Discuss the advantages of self-attention, such as parallelization, long-range dependency handling, and interpretability. Also, address its limitations, like computational complexity and memory usage.'), Task(id=4, title='Self-Attention in Transformers: The Big Picture', brief='Explain how self-attention fi

In [14]:
result["plan"]

Plan(blog_title='Trustworthy Agentic AI Systems: A Cross-Layer Review of Architectures, Threat Models, and Governance Strategies', tasks=[Task(id=1, title='Introduction to Agentic AI Systems', brief='Define what agentic AI systems are, their significance in modern technology, and why trustworthiness is a critical concern. Highlight the growing adoption of AI agents in various sectors like healthcare, finance, and autonomous systems.'), Task(id=2, title='Layered Architecture of Agentic AI Systems', brief='Explore the cross-layer architecture of agentic AI systems. Break it down into key layers such as perception, decision-making, memory, learning, and interaction. Explain the role of each layer and how they interact to form a cohesive system.'), Task(id=3, title='Threat Models in Agentic AI Systems', brief='Dive into the potential threats and vulnerabilities in agentic AI systems. Categorize threats based on layers (e.g., adversarial attacks on perception, data poisoning in learning, or

In [13]:
result

{'topic': 'Trustworthy Agentic AI Systems: A Cross-Layer Review of Architectures, Threat Models, and Governance Strategies',
 'plan': Plan(blog_title='Trustworthy Agentic AI Systems: A Cross-Layer Review of Architectures, Threat Models, and Governance Strategies', tasks=[Task(id=1, title='Introduction to Agentic AI Systems', brief='Define what agentic AI systems are, their significance in modern technology, and why trustworthiness is a critical concern. Highlight the growing adoption of AI agents in various sectors like healthcare, finance, and autonomous systems.'), Task(id=2, title='Layered Architecture of Agentic AI Systems', brief='Explore the cross-layer architecture of agentic AI systems. Break it down into key layers such as perception, decision-making, memory, learning, and interaction. Explain the role of each layer and how they interact to form a cohesive system.'), Task(id=3, title='Threat Models in Agentic AI Systems', brief='Dive into the potential threats and vulnerabilit